    # Functions for calculations
def calculate_ema(stock_df, window):
    return stock_df['Close'].ewm(span=window, adjust=False).mean()

def backtest_crossover_strategy(stock_df, short_window, long_window):
    short_ema = calculate_ema(stock_df, short_window)
    long_ema = calculate_ema(stock_df, long_window)

    # Buy signal: when short EMA crosses above long EMA
    stock_df['Buy Signal'] = (short_ema > long_ema) & (short_ema.shift(1) <= long_ema.shift(1))

    # Sell signal: when short EMA crosses below long EMA
    stock_df['Sell Signal'] = (short_ema < long_ema) & (short_ema.shift(1) >= long_ema.shift(1))

    # Assume starting with cash, so the first signal should be a buy
    initial_position = "cash"
    positions = [initial_position]

    for i in range(1, len(stock_df)):
        if stock_df['Buy Signal'].iloc[i]:
            positions.append("stock")
        elif stock_df['Sell Signal'].iloc[i]:
            positions.append("cash")
        else:
            positions.append(positions[-1])

    stock_df['Position'] = positions
    
    # Calculate returns
    stock_df['Daily Strategy Return'] = np.where(stock_df['Position'] == "stock", stock_df['Close'].pct_change(), 0)
    total_strategy_return = (stock_df['Daily Strategy Return'] + 1).prod()
    return total_strategy_return
    

# test strategy    
short_window = 40  # or any value you'd like to use
long_window = 100  # or any value you'd like to use

total_strategy_return = backtest_crossover_strategy(stock_df, short_window, long_window)
stock_df.to_csv('ema_crossover.csv', index=False)
total_strategy_return = 100*(total_strategy_return-1)
print(f"Total return of EMA Crossover Strategy: {total_strategy_return:.2f}%")